In [27]:
from functools import lru_cache
from textwrap import dedent
import openai
import instructor
from pydantic import BaseModel, Field
from tenacity import retry, stop_after_attempt, wait_fixed
import time
import base64
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env")  # take environment variables from ../.env.

client = instructor.patch(openai.OpenAI(), mode=instructor.Mode.MD_JSON)

class ImageAnalysis(BaseModel):
    title: str = Field(
        ...,
        description="The title of the image.",
    )

    description: str = Field(
        ...,
        description="The description of the image. Include any summary that can help someone find the image in a database.",
    )
    markdown_notes: str = Field(
        ...,
        description="Makedown notes to understand the image better.",
    )
    markdown_tables: str = Field(
        ...,
        description="A markdown table listing the features of the image.",
    )

    def display(self):
        from IPython.display import Markdown, display

        report = f"## {self.title} \n{self.description} \n{self.markdown_notes} \n\n{self.markdown_tables}"
        display(Markdown(dedent(report)))

def analyse_image(img) -> ImageAnalysis:
    with open(img, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    image_url = f"data:image/jpeg;base64,{encoded_string}"
    print("Processing image: ", img)
    resp = client.chat.completions.create(
        model="gpt-4-vision-preview",
        max_tokens=4096,
        max_retries=2,
        response_model=ImageAnalysis,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": """
                You are a world-class analyst tasked with analyzing images. 
                Your goal is to generate a comprehensive summary that makes the image easily searchable in a database.
                """
            },
            {
                "role": "user",
                "content": [
                    f"Return a detailed analysis of the image, adhering to the structure defined by {ImageAnalysis.model_json_schema()}",
                    *[{"type": "image_url", "image_url": image_url}],
                ],
            },
        ],
    )
    return resp

In [26]:
processed_image = analyse_image("img2.jpg")
processed_image.display()


## Aerial View of a Parking Lot 
The image is an aerial shot of a parking lot filled with vehicles. The vehicles are arranged in neat rows, showcasing a variety of colors and models. The parking lot surface is a light gray, with the lines for parking spaces faintly visible. The image captures the organization and density of urban vehicle storage in a confined space. 
### Aerial View of a Parking Lot
- **Perspective**: Aerial, top-down view
- **Content**: Vehicles parked in rows
- **Colors**: Various, including red, blue, white, black, and gray
- **Patterns**: Repetitive arrangement of vehicles
- **Surface**: Light gray asphalt with parking space lines
- **Usage**: Can be used to illustrate urban planning, parking lot design, or vehicle density in urban areas. 

| Feature | Description |
| --- | --- |
| Perspective | Aerial, top-down view |
| Vehicle Colors | Red, blue, white, black, gray |
| Parking Lot Surface | Light gray asphalt |
| Vehicle Arrangement | Neat rows |
| Image Theme | Urban vehicle storage, parking lot design |

In [31]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(processed_image.model_dump())


{   'description': 'The image captures an aerial view of a beach scene with a '
                   'wooden pier extending into the sea. People are seen '
                   'swimming in the water and lounging on the beach with '
                   'various colorful beach umbrellas and towels. The contrast '
                   'between the turquoise water, sandy beach, and the brown '
                   'pier creates a visually appealing composition.',
    'markdown_notes': '### Aerial Beach Scene\n'
                      '- **Pier**: Central wooden structure extending into the '
                      'sea\n'
                      '- **Beachgoers**: Individuals visible on the pier and '
                      'beach\n'
                      '- **Swimmers**: People in the water near the pier\n'
                      '- **Beach Umbrellas**: Colorful umbrellas dotting the '
                      'beach\n'
                      '- **Beach Towels**: Assorted towels on the sand\n'
            

In [28]:
processed_image = analyse_image("img3.jpg")
processed_image.display()



Processing image:  img3.jpg


## Aerial View of a Beach Pier 
The image captures an aerial view of a beach scene with a wooden pier extending into the sea. People are seen swimming in the water and lounging on the beach with various colorful beach umbrellas and towels. The contrast between the turquoise water, sandy beach, and the brown pier creates a visually appealing composition. 
### Aerial Beach Scene
- **Pier**: Central wooden structure extending into the sea
- **Beachgoers**: Individuals visible on the pier and beach
- **Swimmers**: People in the water near the pier
- **Beach Umbrellas**: Colorful umbrellas dotting the beach
- **Beach Towels**: Assorted towels on the sand
- **Water Texture**: Clear turquoise sea with visible sand underneath
- **Beach Sand**: Light-colored sand with footprints and beachwear 

| Feature | Description |
| --- | --- |
| Pier | Wooden, central in the image, extending into the sea |
| People | Beachgoers on the pier and sand, swimmers in the water |
| Beach Umbrellas | Various colors, providing shade for beachgoers |
| Beach Towels | Scattered across the sand, different colors and patterns |
| Sea | Turquoise water, clear with visible sand beneath |
| Sand | Light-colored with footprints and beachwear items |